In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math
# import pixiedust                   ##debuger
# import glob, os

In [2]:
all_stim = pd.read_csv('2018_02_21_10_06_10_stim.csv', sep=';')

In [3]:
responses = pd.read_csv('2018_02_21_10_06_10.csv', sep=';', encoding = "ISO-8859-1")

# Feature task

In [4]:
#clean feature response with "bad response"
resp_feat = responses[responses["FeatConj"] == 0]
bad_feat = resp_feat.loc[(resp_feat['stType'] == ' distractor2 ') | (resp_feat['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_feat = resp_feat[~resp_feat['trialN']
          .isin(bad_feat)]
resp_feat = resp_feat.reset_index(drop=True)
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,col,row,touchX,touchY,touchDist,targDist,touchN,error,switches,changeScreen
0,2,49,m,0,0,r/g,0,40,50,0.0,...,0.0,7.0,128.5,672.5,0.0,0.000000,1,0,2,2
1,2,49,m,0,0,r/g,0,40,50,0.0,...,1.0,6.0,212.0,588.0,115.277925033373,113.523958,2,0,1,2
2,2,49,m,0,0,r/g,0,40,50,0.0,...,2.0,6.0,286.5,588.0,72.5844335928855,79.582466,3,0,1,2
3,2,49,m,0,0,r/g,0,40,50,0.0,...,0.0,4.0,127.5,418.5,230.39151460069,257.802000,4,0,0,2
4,2,49,m,0,0,r/g,0,40,50,0.0,...,0.0,3.0,125.0,351.0,64.5077514722068,48.387227,5,0,1,2


In [5]:
resp_feat1 = resp_feat.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_feat1['pos_x_r'] = np.around(resp_feat1['pos_x_r'], decimals=2)
resp_feat1['pos_y_r'] = np.around(resp_feat1['pos_y_r'], decimals=2)
resp_feat['resp_pos'] = resp_feat1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_feat1['pos_r'] = list(zip(resp_feat1['pos_x_r'], resp_feat1['pos_y_r']))

resp_feat_c = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                                'pos_x_r',
                                                                'pos_y_r'])]

resp_feat1 = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_feat1 = resp_feat1.reset_index(drop=True)
resp_feat1.type = resp_feat1.type.str.strip()

In [6]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r
0,2,0,target2,120.22,672.86,"(120.22, 672.86)"
1,2,0,target1,205.81,598.29,"(205.81, 598.29)"
2,2,0,target2,285.30,594.37,"(285.3, 594.37)"
3,2,0,target2,110.68,404.71,"(110.68, 404.71)"
4,2,0,target1,120.28,357.29,"(120.28, 357.29)"
...,...,...,...,...,...,...
195,2,9,target2,144.96,192.27,"(144.96, 192.27)"
196,2,9,target2,102.56,399.26,"(102.56, 399.26)"
197,2,9,target2,119.66,487.26,"(119.66, 487.26)"
198,2,9,target1,280.55,516.68,"(280.55, 516.68)"


## Strict line funcs

In [7]:
def area(x1, y1, x2, y2, x3, y3): 
    
    return abs((x1 * (y2 - y3) + 
                x2 * (y3 - y1) + 
                x3 * (y1 - y2)) / 2.0)

In [8]:
def check_area(x1, y1, x2, y2, lst1, lst2):
    r = 15
    e = -1
    #define lines coefficients
    m = (y1 - y2)/(x1-x2)
    b = y1 - m*x1
    if m != 0:
        m_c = e/m
    else:
        m_c = 0
    b1 = y1 - m_c*x1
    b2 = y2 - m_c*x2
    #define check points
    R = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    kx = (r/R) * (y2 - y1)
    ky = (r/R) * (x2 - x1)
    x1_check1 = x1 - kx
    y1_check1 = y1 + ky
    x1_check2 = x1 + kx
    y1_check2 = y1 - ky
    x2_check1 = x2 - kx
    y2_check1 = y2 + ky
    x2_check2 = x2 + kx
    y2_check2 = y2 - ky
    #check if the point inside the rectangle
    final_list= []
    for i in range(0, len(lst1)):#area of triangles and rectangle
        s1 = area(lst1[i], lst2[i], x1_check1, y1_check1, x1_check2, y1_check2)
        s2 = area(lst1[i], lst2[i], x1_check2, y1_check2, x2_check2, y2_check2)
        s3 = area(lst1[i], lst2[i], x2_check2, y2_check2, x2_check1, y2_check1)
        s4 = area(lst1[i], lst2[i], x2_check1, y2_check1, x1_check1, y1_check1)
        s_tr = round((s1 + s2 + s3 + s4), 3)
        s_rec = round((r * 2 * R), 3)
        #final check
        if s_tr == s_rec:
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list  
            

In [9]:
check_area(0, 0, 1, 1, [0,0], [0.5,2])

[1, 1]

In [10]:
len([5,3,3])

3

### Add x1,x2,y1,y2 (I was too sleepy...)

In [11]:
resp_feat_c['x1'] = resp_feat_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_feat_c['x2'] = resp_feat_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_feat_c['y1'] = resp_feat_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_feat_c['y2'] = resp_feat_c['pos_y_r'].apply(lambda x: x - 106.5)

### Distances

In [12]:
feat_r = resp_feat1.values.tolist()

In [13]:
#only useful df of all stimuli for feature task
all_feat = all_stim[all_stim["FeatConj"] == 0]
feat_target = all_feat[(all_feat['stType'] == 'target1') 
                       | (all_feat['stType'] == 'target2')]
feat_distr = all_feat[(all_feat['stType'] == 'distractor1') 
                      | (all_feat['stType'] == 'distractor2')]

In [14]:
#useful targets for feature (only 3 columns)
feat_rest_t = feat_target[~feat_target['trialN']
          .isin(bad_feat)]
feat_rest_t = feat_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
feat_rest_t['pos_x_t'] = np.around(feat_rest_t['pos_x_t'], decimals=2)
feat_rest_t['pos_y_t'] = np.around(feat_rest_t['pos_y_t'], decimals=2)
feat_rest_t['pos_t'] = list(zip(feat_rest_t['pos_x_t'], feat_rest_t['pos_y_t']))
feat_rest_t = feat_rest_t.loc[:, feat_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
feat_rest_t = feat_rest_t.reset_index(drop=True)

In [15]:
#numbers of trials in list
trials = feat_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [16]:
participant = feat_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [17]:
dfs_tar_f = dict(tuple(feat_rest_t.groupby('target')))
feat_t1 = dfs_tar_f['target1']
feat_t2 = dfs_tar_f['target2']

In [18]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [19]:
def closest_node(node, nodes):
    indexArr = np.argwhere(nodes == node)
    nodes = np.delete(nodes,indexArr, 0)
    closest_index = distance.cdist([node], nodes, 'euclidean').argmin()
    return nodes[closest_index]

In [20]:
# %%pixie_debugger 
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1
               

In [21]:
resp_feat['closest_same_target'] = closest_same

In [22]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target2':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [23]:
resp_feat['closest_other_taget'] = closest_other

In [24]:
feat_rest_d = feat_distr[~feat_distr['trialN']
          .isin(bad_feat)]
feat_rest_d = feat_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
feat_rest_d['pos_x_d'] = np.around(feat_rest_d['pos_x_d'], decimals=2)
feat_rest_d['pos_y_d'] = np.around(feat_rest_d['pos_y_d'], decimals=2)
feat_rest_d['pos_d'] = list(zip(feat_rest_d['pos_x_d'], feat_rest_d['pos_y_d']))
feat_rest_d = feat_rest_d.loc[:, feat_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
feat_rest_d = feat_rest_d.reset_index(drop=True)

In [25]:
dfs_d_f = dict(tuple(feat_rest_d.groupby('distractor')))
feat_d1 = dfs_d_f['distractor1']
feat_d2 = dfs_d_f['distractor2']
d1 = dict(tuple(feat_d1.groupby('trial_d')))
d2 = dict(tuple(feat_d2.groupby('trial_d')))

In [26]:
# closest blue dot
closest_same_d = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    closest_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_same_d.append(closest_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [27]:
resp_feat['closest_blue_dot'] = closest_same_d

In [28]:
# closest yellow dot
closest_d2 = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    c_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_d2.append(c_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [29]:
resp_feat['closest_yellow_dot'] = closest_d2

# Crowding_feat

In [30]:
def close_or_not(x1, x2, y1, y2, lst1, lst2):
    count = 0
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

In [31]:
def close_or_not_new(x, y, lst1, lst2):
    count = 0
    x1 = x + 108.6
    x2 = x - 108.6
    y1 = y + 106.5
    y2 = y - 106.5
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

## Crowding around selected target

In [32]:
feat_r = resp_feat_c.values.tolist()

In [33]:
feat_t1[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index) 
feat_t2[['pos_x', 'pos_y']] = pd.DataFrame(feat_t2['pos_t'].tolist(), index=feat_t1.index)

In [34]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [35]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [36]:
resp_feat['crowd_selected'] = crowding

In [37]:
resp_feat

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,touchN,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd_selected
0,2,49,m,0,0,r/g,0,40,50,0.0,...,1,0,2,2,"[120.22, 672.86]","[285.3, 594.37]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0
1,2,49,m,0,0,r/g,0,40,50,0.0,...,2,0,1,2,"[205.81, 598.29]","[451.58, 595.88]","[285.3, 594.37]","[233.0, 322.97]","[467.91, 663.78]",1
2,2,49,m,0,0,r/g,0,40,50,0.0,...,3,0,1,2,"[285.3, 594.37]","[120.22, 672.86]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0
3,2,49,m,0,0,r/g,0,40,50,0.0,...,4,0,0,2,"[110.68, 404.71]","[109.76, 266.01]","[120.28, 357.29]","[233.0, 322.97]","[318.47, 204.98]",0
4,2,49,m,0,0,r/g,0,40,50,0.0,...,5,0,1,2,"[120.28, 357.29]","[308.05, 128.76]","[110.68, 404.71]","[233.0, 322.97]","[318.47, 204.98]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,49,m,9,0,r/g,0,40,50,0.0,...,16,0,1,0,"[144.96, 192.27]","[119.66, 487.26]","[205.84, 178.18]","[228.49, 498.79]","[152.58, 280.36]",0
196,2,49,m,9,0,r/g,0,40,50,0.0,...,20,0,0,0,"[102.56, 399.26]","[119.66, 487.26]","[280.55, 516.68]","[228.49, 498.79]","[152.58, 280.36]",0
197,2,49,m,9,0,r/g,0,40,50,0.0,...,21,0,0,0,"[119.66, 487.26]","[409.72, 494.06]","[280.55, 516.68]","[140.56, 589.97]","[235.17, 438.17]",0
198,2,49,m,9,0,r/g,0,40,50,0.0,...,22,0,1,0,"[280.55, 516.68]","[480.82, 645.43]","[409.72, 494.06]","[228.49, 498.79]","[235.17, 438.17]",1


## Crowding closest same

In [38]:
feat_cl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                   'trialN',
                                                           'stType',
                                                                   'closest_same_target',
                                                                  'closest_other_taget'])]

In [39]:
feat_cl[['same_x', 'same_y']] = pd.DataFrame(feat_cl['closest_same_target'].tolist(), index=feat_cl.index) 
feat_cl[['switch_x', 'switch_y']] = pd.DataFrame(feat_cl['closest_other_taget'].tolist(), index=feat_cl.index)

In [40]:
feat_clos = feat_cl.values.tolist()

In [41]:
crowding_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_clos)):
            if part == feat_clos[j][0]:
                if n_tr == feat_clos[j][1]:
                    if feat_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(feat_clos[j][5],
                                                      feat_clos[j][6],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_same.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(feat_clos[j][5],
                                                  feat_clos[j][6],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_same.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [42]:
resp_feat['crowd_same'] = crowding_same

## Crowding closest switch

In [43]:
crowding_switch = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_clos)):
            if part == feat_clos[j][0]:
                if n_tr == feat_clos[j][1]:
                    if feat_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(feat_clos[j][7],
                                                      feat_clos[j][8],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_switch.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(feat_clos[j][7],
                                                  feat_clos[j][8],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_switch.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [44]:
resp_feat['crowd_switch'] = crowding_switch

# Feature strict line

In [45]:
all_feat_sl = all_feat.loc[:, all_feat.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [46]:
targ = dict(tuple(all_feat_sl.groupby('stType')))
feat_t1 = targ['target1']
feat_t2 = targ['target2']

In [47]:
t1 = dict(tuple(feat_t1.groupby('trialN')))
t2 = dict(tuple(feat_t2.groupby('trialN')))

In [48]:
resp_feat_sl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [49]:
resp_feat_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_feat_sl['closest_same_target']
                                              .tolist(), index=resp_feat_sl.index) 
resp_feat_sl[['r_x', 'r_y']] = pd.DataFrame(resp_feat_sl['resp_pos']
                                              .tolist(), index=resp_feat_sl.index) 

In [50]:
trials = all_feat_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [51]:
feat_r_sl = resp_feat_sl.values.tolist()

In [52]:
resp_feat_sl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [53]:
resp_feat_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_feat_sl['closest_same_target']
                                              .tolist(), index=resp_feat_sl.index) 
resp_feat_sl[['r_x', 'r_r']] = pd.DataFrame(resp_feat_sl['resp_pos']
                                              .tolist(), index=resp_feat_sl.index) 

In [54]:
resp_feat_sl

,participantN,trialN,stType,resp_pos,closest_same_target,cl_x,cl_y,r_x,r_r
0,2,0,target2,"[120.22, 672.86]","[285.3, 594.37]",285.30,594.37,120.22,672.86
1,2,0,target1,"[205.81, 598.29]","[451.58, 595.88]",451.58,595.88,205.81,598.29
2,2,0,target2,"[285.3, 594.37]","[120.22, 672.86]",120.22,672.86,285.30,594.37
3,2,0,target2,"[110.68, 404.71]","[109.76, 266.01]",109.76,266.01,110.68,404.71
4,2,0,target1,"[120.28, 357.29]","[308.05, 128.76]",308.05,128.76,120.28,357.29
...,...,...,...,...,...,...,...,...,...
195,2,9,target2,"[144.96, 192.27]","[119.66, 487.26]",119.66,487.26,144.96,192.27
196,2,9,target2,"[102.56, 399.26]","[119.66, 487.26]",119.66,487.26,102.56,399.26
197,2,9,target2,"[119.66, 487.26]","[409.72, 494.06]",409.72,494.06,119.66,487.26
198,2,9,target1,"[280.55, 516.68]","[480.82, 645.43]",480.82,645.43,280.55,516.68


In [55]:
trials = resp_feat_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [56]:
feat_r_sl = resp_feat_sl.values.tolist()

In [57]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['posX'].values.tolist()
        t1_arr_y = t1.get(trials[i])['posY'].values.tolist()
        t2_arr_x = t2.get(trials[i])['posX'].values.tolist()
        t2_arr_y = t2.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(feat_r_sl)):
            if part == feat_r_sl[j][0]:
                if n_tr == feat_r_sl[j][1]:
                    if feat_clos[j][2] == 'target1':
                        strict = check_area(feat_r_sl[j][7], 
                                            feat_r_sl[j][8],
                                            feat_r_sl[j][5], 
                                            feat_r_sl[j][6],
                                            t2_arr_x, t2_arr_y)
                        strict_line.append(strict)
                    else:
                        strict = check_area(feat_r_sl[j][7], 
                                            feat_r_sl[j][8],
                                            feat_r_sl[j][5], 
                                            feat_r_sl[j][6],
                                            t1_arr_x, t1_arr_y)
                        strict_line.append(strict)
                        
            else:
                i += 1
        else:
            n += 1

In [58]:
can_provide_line = []
for i in range(0, len(strict_line)):
        if 1 in strict_line[i]:
            can_provide_line.append(1)
        else:
            can_provide_line.append(0)
    

In [59]:
resp_feat['strict_line'] = can_provide_line

In [60]:
resp_feat

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd_selected,crowd_same,crowd_switch,strict_line
0,2,49,m,0,0,r/g,0,40,50,0.0,...,2,"[120.22, 672.86]","[285.3, 594.37]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0,0,0,0
1,2,49,m,0,0,r/g,0,40,50,0.0,...,2,"[205.81, 598.29]","[451.58, 595.88]","[285.3, 594.37]","[233.0, 322.97]","[467.91, 663.78]",1,1,1,1
2,2,49,m,0,0,r/g,0,40,50,0.0,...,2,"[285.3, 594.37]","[120.22, 672.86]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0,0,0,0
3,2,49,m,0,0,r/g,0,40,50,0.0,...,2,"[110.68, 404.71]","[109.76, 266.01]","[120.28, 357.29]","[233.0, 322.97]","[318.47, 204.98]",0,0,0,1
4,2,49,m,0,0,r/g,0,40,50,0.0,...,2,"[120.28, 357.29]","[308.05, 128.76]","[110.68, 404.71]","[233.0, 322.97]","[318.47, 204.98]",1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,49,m,9,0,r/g,0,40,50,0.0,...,0,"[144.96, 192.27]","[119.66, 487.26]","[205.84, 178.18]","[228.49, 498.79]","[152.58, 280.36]",0,0,0,0
196,2,49,m,9,0,r/g,0,40,50,0.0,...,0,"[102.56, 399.26]","[119.66, 487.26]","[280.55, 516.68]","[228.49, 498.79]","[152.58, 280.36]",0,0,0,0
197,2,49,m,9,0,r/g,0,40,50,0.0,...,0,"[119.66, 487.26]","[409.72, 494.06]","[280.55, 516.68]","[140.56, 589.97]","[235.17, 438.17]",0,0,0,0
198,2,49,m,9,0,r/g,0,40,50,0.0,...,0,"[280.55, 516.68]","[480.82, 645.43]","[409.72, 494.06]","[228.49, 498.79]","[235.17, 438.17]",1,1,0,0


In [61]:
resp_feat['blocked_crowd'] = [1 if x > 0 & y>0 else 0 
                              for x, y in zip(resp_feat['crowd_same'],resp_feat['strict_line'])]

In [62]:
resp_feat['blocked_crowd'].unique()

array([0], dtype=int64)

# Conjunction task

In [63]:
resp_conj = responses[responses["FeatConj"] == 1]
bad_conj = resp_conj.loc[(resp_conj['stType'] == ' distractor2 ') | (resp_conj['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_conj = resp_conj[~resp_conj['trialN']
          .isin(bad_conj)]
resp_conj = resp_conj.reset_index(drop=True)

In [64]:
resp_conj1 = resp_conj.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_conj1['pos_x_r'] = np.around(resp_conj1['pos_x_r'], decimals=2)
resp_conj1['pos_y_r'] = np.around(resp_conj1['pos_y_r'], decimals=2)
resp_conj['resp_pos'] = resp_conj1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_conj1['pos_r'] = list(zip(resp_conj1['pos_x_r'], resp_conj1['pos_y_r']))

resp_conj_c = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                               'pos_x_r',
                                                               'pos_y_r'])]

resp_conj1 = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_conj1 = resp_conj1.reset_index(drop=True)
resp_conj1.type = resp_conj1.type.str.strip()

### Add x1,x2,y1,y2

In [65]:
resp_conj_c['x1'] = resp_conj_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_conj_c['x2'] = resp_conj_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_conj_c['y1'] = resp_conj_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_conj_c['y2'] = resp_conj_c['pos_y_r'].apply(lambda x: x - 106.5)

### Distances

In [66]:
conj_r = resp_conj1.values.tolist()
## [prtN, trialN, target_type, pos]

In [67]:
#only useful df of all stimuli for conjunction task
all_conj = all_stim[all_stim["FeatConj"] == 1]
conj_target = all_conj[(all_conj['stType'] == 'target1') |
                       (all_conj['stType'] == 'target2')]
conj_distr = all_conj[(all_conj['stType'] == 'distractor1') |
                      (all_conj['stType'] == 'distractor2')]

In [68]:
#useful targets for conjunction
conj_rest_t = conj_target[~conj_target['trialN']
          .isin(bad_conj)]
conj_rest_t = conj_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
conj_rest_t['pos_x_t'] = np.around(conj_rest_t['pos_x_t'], decimals=2)
conj_rest_t['pos_y_t'] = np.around(conj_rest_t['pos_y_t'], decimals=2)
conj_rest_t['pos_t'] = list(zip(conj_rest_t['pos_x_t'], conj_rest_t['pos_y_t']))
conj_rest_t = conj_rest_t.loc[:, conj_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
conj_rest_t = conj_rest_t.reset_index(drop=True)

In [69]:
trials = conj_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [70]:
participant = conj_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [71]:
dfs_tar_c = dict(tuple(conj_rest_t.groupby('target')))
conj_t1 = dfs_tar_c['target1']
conj_t2 = dfs_tar_c['target2']

In [72]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [73]:
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [74]:
resp_conj['closest_same_target'] = closest_same

In [75]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [76]:
resp_conj['closest_other_target'] = closest_other

In [77]:
#useful distractors for conjunction
conj_rest_d = conj_distr[~conj_distr['trialN']
          .isin(bad_conj)]
conj_rest_d = conj_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
conj_rest_d['pos_x_d'] = np.around(conj_rest_d['pos_x_d'], decimals=2)
conj_rest_d['pos_y_d'] = np.around(conj_rest_d['pos_y_d'], decimals=2)
conj_rest_d['pos_d'] = list(zip(conj_rest_d['pos_x_d'], conj_rest_d['pos_y_d']))
conj_rest_d = conj_rest_d.loc[:, conj_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
conj_rest_d = conj_rest_d.reset_index(drop=True)

In [78]:
dfs_d_c = dict(tuple(conj_rest_d.groupby('distractor')))
conj_d1 = dfs_d_c['distractor1']
conj_d2 = dfs_d_c['distractor2']
d1 = dict(tuple(conj_d1.groupby('trial_d')))
d2 = dict(tuple(conj_d2.groupby('trial_d')))

In [79]:
same_color = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_d1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_color.append(closest_d1.tolist())         
                    else:
                        closest_d2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_color.append(closest_d2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [80]:
resp_conj['closest_dist_same_color'] = same_color

In [81]:
same_shape = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_shape.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_shape.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [82]:
resp_conj['closest_dist_same_shape'] = same_shape

# Crowding_conj

In [83]:
conj_r = resp_conj_c.values.tolist()

In [84]:
conj_t1[['pos_x', 'pos_y']] = pd.DataFrame(conj_t1['pos_t'].tolist(), index=conj_t1.index) 
conj_t2[['pos_x', 'pos_y']] = pd.DataFrame(conj_t2['pos_t'].tolist(), index=conj_t1.index)

In [85]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [86]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [87]:
resp_conj['crowd'] = crowding

## Crowding closest same

In [88]:
conj_cl = resp_conj.loc[:, resp_conj.columns.intersection(['participantN',
                                                                   'trialN',
                                                           'stType',
                                                                   'closest_same_target',
                                                                  'closest_other_target'])]

In [89]:
conj_cl[['same_x', 'same_y']] = pd.DataFrame(conj_cl['closest_same_target'].tolist(), index=conj_cl.index) 
conj_cl[['switch_x', 'switch_y']] = pd.DataFrame(conj_cl['closest_other_target'].tolist(), index=conj_cl.index)

In [90]:
conj_clos = conj_cl.values.tolist()

In [91]:
crowding_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_clos)):
            if part == conj_clos[j][0]:
                if n_tr == conj_clos[j][1]:
                    if conj_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(conj_clos[j][5],
                                                      conj_clos[j][6],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_same.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(conj_clos[j][5],
                                                  conj_clos[j][6],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_same.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [92]:
resp_conj['crowd_same'] = crowding_same

## Crowding closest switch

In [93]:
crowding_switch = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_clos)):
            if part == conj_clos[j][0]:
                if n_tr == conj_clos[j][1]:
                    if conj_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(conj_clos[j][7],
                                                      conj_clos[j][8],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_switch.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(conj_clos[j][7],
                                                  conj_clos[j][8],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_switch.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [94]:
resp_conj['crowd_switch'] = crowding_switch

## Strict line conj

In [95]:
all_conj_sl = all_conj.loc[:, all_conj.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [96]:
targ = dict(tuple(all_conj_sl.groupby('stType')))
conj_t1 = targ['target1']
conj_t2 = targ['target2']

In [97]:
t1 = dict(tuple(conj_t1.groupby('trialN')))
t2 = dict(tuple(conj_t2.groupby('trialN')))

In [98]:
resp_conj_sl = resp_conj.loc[:, resp_conj.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [99]:
resp_conj_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_conj_sl['closest_same_target']
                                              .tolist(), index=resp_conj_sl.index) 
resp_conj_sl[['r_x', 'r_y']] = pd.DataFrame(resp_conj_sl['resp_pos']
                                              .tolist(), index=resp_conj_sl.index) 

In [100]:
resp_conj_sl

,participantN,trialN,stType,resp_pos,closest_same_target,cl_x,cl_y,r_x,r_y
0,2,0,target2,"[743.96, 95.48]","[565.77, 327.1]",565.77,327.10,743.96,95.48
1,2,0,target1,"[796.89, 357.27]","[905.66, 411.54]",905.66,411.54,796.89,357.27
2,2,0,target1,"[905.66, 411.54]","[796.89, 357.27]",796.89,357.27,905.66,411.54
3,2,0,target2,"[879.56, 509.54]","[534.68, 402.25]",534.68,402.25,879.56,509.54
4,2,0,target1,"[893.39, 663.17]","[710.97, 578.09]",710.97,578.09,893.39,663.17
...,...,...,...,...,...,...,...,...,...
195,2,9,target2,"[906.43, 117.28]","[716.84, 484.01]",716.84,484.01,906.43,117.28
196,2,9,target2,"[748.5, 99.32]","[906.43, 117.28]",906.43,117.28,748.50,99.32
197,2,9,target2,"[893.4, 481.44]","[716.84, 484.01]",716.84,484.01,893.40,481.44
198,2,9,target2,"[716.84, 484.01]","[633.14, 596.99]",633.14,596.99,716.84,484.01


In [101]:
trials = all_conj_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [102]:
conj_r_sl = resp_conj_sl.values.tolist()

In [103]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['posX'].values.tolist()
        t1_arr_y = t1.get(trials[i])['posY'].values.tolist()
        t2_arr_x = t2.get(trials[i])['posX'].values.tolist()
        t2_arr_y = t2.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(conj_r_sl)):
            if part == conj_r_sl[j][0]:
                if n_tr == conj_r_sl[j][1]:
                    if conj_clos[j][2] == 'target1':
                        strict = check_area(conj_r_sl[j][7], 
                                            conj_r_sl[j][8],
                                            conj_r_sl[j][5], 
                                            conj_r_sl[j][6],
                                            t2_arr_x, t2_arr_y)
                        strict_line.append(strict)
                    else:
                        strict = check_area(conj_r_sl[j][7], 
                                            conj_r_sl[j][8],
                                            conj_r_sl[j][5], 
                                            conj_r_sl[j][6],
                                            t1_arr_x, t1_arr_y)
                        strict_line.append(strict)
                        
            else:
                i += 1
        else:
            n += 1

In [104]:
can_provide_line = []
for i in range(0, len(strict_line)):
    if 1 in strict_line[i]:
        can_provide_line.append(1)
    else:
        can_provide_line.append(0)
    

In [105]:
resp_conj['strict_line'] = can_provide_line

In [106]:
resp_conj['blocked_crowd'] = [1 if x > 0 & y > 0 else 0 
                              for x, y in zip(resp_conj['crowd_same'],resp_conj['strict_line'])]

# Make csv

In [108]:
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd_selected,crowd_same,crowd_switch,strict_line,blocked_crowd
0,2,49,m,0,0,r/g,0,40,50,0.0,...,"[120.22, 672.86]","[285.3, 594.37]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0,0,0,0,0
1,2,49,m,0,0,r/g,0,40,50,0.0,...,"[205.81, 598.29]","[451.58, 595.88]","[285.3, 594.37]","[233.0, 322.97]","[467.91, 663.78]",1,1,1,1,0
2,2,49,m,0,0,r/g,0,40,50,0.0,...,"[285.3, 594.37]","[120.22, 672.86]","[205.81, 598.29]","[233.0, 322.97]","[467.91, 663.78]",0,0,0,0,0
3,2,49,m,0,0,r/g,0,40,50,0.0,...,"[110.68, 404.71]","[109.76, 266.01]","[120.28, 357.29]","[233.0, 322.97]","[318.47, 204.98]",0,0,0,1,0
4,2,49,m,0,0,r/g,0,40,50,0.0,...,"[120.28, 357.29]","[308.05, 128.76]","[110.68, 404.71]","[233.0, 322.97]","[318.47, 204.98]",1,1,1,1,0


In [109]:
resp_conj

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,resp_pos,closest_same_target,closest_other_target,closest_dist_same_color,closest_dist_same_shape,crowd,crowd_same,crowd_switch,strict_line,blocked_crowd
0,2,49,m,0,0,rs/gd,1,40,50,0.0,...,"[743.96, 95.48]","[565.77, 327.1]","[538.29, 200.83]","[490.24, 185.96]","[699.83, 180.13]",0,0,0,0,0
1,2,49,m,0,0,rs/gd,1,40,50,0.0,...,"[796.89, 357.27]","[905.66, 411.54]","[879.56, 509.54]","[743.46, 358.26]","[798.19, 480.71]",1,1,1,0,0
2,2,49,m,0,0,rs/gd,1,40,50,0.0,...,"[905.66, 411.54]","[796.89, 357.27]","[879.56, 509.54]","[905.12, 579.24]","[921.42, 326.0]",1,1,1,0,0
3,2,49,m,0,0,rs/gd,1,40,50,0.0,...,"[879.56, 509.54]","[534.68, 402.25]","[905.66, 411.54]","[798.19, 480.71]","[905.12, 579.24]",0,0,0,0,0
4,2,49,m,0,0,rs/gd,1,40,50,0.0,...,"[893.39, 663.17]","[710.97, 578.09]","[879.56, 509.54]","[905.12, 579.24]","[798.19, 480.71]",1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,49,m,9,0,rs/gd,1,40,50,0.0,...,"[906.43, 117.28]","[716.84, 484.01]","[830.97, 125.63]","[804.32, 182.02]","[889.04, 350.47]",0,0,0,0,0
196,2,49,m,9,0,rs/gd,1,40,50,0.0,...,"[748.5, 99.32]","[906.43, 117.28]","[830.97, 125.63]","[804.32, 182.02]","[734.45, 334.84]",0,0,0,0,0
197,2,49,m,9,0,rs/gd,1,40,50,0.0,...,"[893.4, 481.44]","[716.84, 484.01]","[830.97, 125.63]","[874.39, 573.6]","[918.14, 430.29]",0,0,0,0,0
198,2,49,m,9,0,rs/gd,1,40,50,0.0,...,"[716.84, 484.01]","[633.14, 596.99]","[575.8, 280.04]","[795.11, 416.68]","[807.42, 507.15]",0,0,0,0,0


In [110]:
#resp_feat.to_csv (r'C:\Users\lttso\Desktop\feature_task.csv', index = False,  mode='a', header=False)

In [111]:
#resp_conj.to_csv (r'C:\Users\lttso\Desktop\conjunction_task.csv', index = False, mode='a', header=False)